# `StableSpectralElements.jl` - 3D linear advection example

First, let's load the required packages.

In [1]:
using Pkg; Pkg.add("SnoopCompile")
using StableSpectralElements, BenchmarkTools, OrdinaryDiffEq, SnoopCompile

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed TypedSyntax ─ v1.2.3
   Installed Cthulhu ───── v2.9.3
    Updating `~/Research/StableSpectralElements.jl/Project.toml`
  [aa65fe97] + SnoopCompile v2.10.8
    Updating `~/Research/StableSpectralElements.jl/Manifest.toml`
  [f68482b8] + Cthulhu v2.9.3
  [08572546] + FlameGraphs v1.0.0
  [1eca21be] + FoldingTrees v1.2.1
  [9b13fd28] + IndirectArrays v1.0.0
  [c3a54625] + JET v0.8.13
  [70703baa] + JuliaSyntax v0.4.6
  [1d6d02ad] + LeftChildRightSiblingTrees v0.2.0
  [aa65fe97] + SnoopCompile v2.10.8
  [e2b509da] + SnoopCompileCore v2.10.0
  [69024149] + StringEncodings v0.3.7
  [d265eb64] + TypedSyntax v1.2.3
  [b8c1c048] + WidthLimitedIO v1.0.1
  [ddb6d928] + YAML v0.4.9
Precompiling project...
  ✓ TypedSyntax
  ✓ Cthulhu
  ✓ SnoopCompile
  ✓ StableSpectralElements
  4 dependencies successfully precompiled in 36 seconds. 335 already precompiled.


We will solve the three-dimensional linear advection equation
$$
\partial_t U(\boldsymbol{x},t) + \nabla_{\boldsymbol{x}} \cdot (\boldsymbol{a} U(\boldsymbol{x},t)) = 0,  \quad \forall\, (\boldsymbol{x}, t) \in \Omega \times (0,T),
$$
with $a = [1,1,1]^{\mathrm{T}}$ and $\Omega = (0,L)^3$, where we take $L = 1$, and $T = 1$, and impose periodic boundary conditions as well as the initial condition
$$
U(\boldsymbol{x},0) = \cos(2\pi x_1 / L)\cos(2\pi x_2 / L)\cos(2\pi x_3 / L), \quad \forall \, \boldsymbol{x} \in \Omega.
$$

In [2]:
a = (1.0,1.0,1.0)  # advection velocity
L = 1.0  # domain length
T = 1.0  # end time

conservation_law = LinearAdvectionEquation(a)
exact_solution = InitialDataCosine(1.0,(2π/L, 2π/L, 2π/L));

We'll discretize the above problem using a `ModalTensor` discretization of degree $p = 7$, on a curved (i.e. using a mapping of degree $p_{\mathrm{map}} = 3$) tetrahedral mesh with $M = 2$ edges in each direction. A skew-symmetric split form will be used to treat the curvilinear coordinate transformation in a provably stable manner, and we will use an upwind (i.e. Lax-Friedrichs) numerical flux.

In [3]:
M = 4
p = 4

reference_approximation = ReferenceApproximation(ModalTensor(p), Tet(), mapping_degree=3);

In [4]:
form = StandardForm(mapping_form=SkewSymmetricMapping(), 
    inviscid_numerical_flux=LaxFriedrichsNumericalFlux())

StandardForm{SkewSymmetricMapping, LaxFriedrichsNumericalFlux, BR1}(SkewSymmetricMapping(), LaxFriedrichsNumericalFlux(0.5), BR1())

In [5]:
uniform_mesh = uniform_periodic_mesh(reference_approximation,
    ((0.0,L),(0.0,L),(0.0,L)), (M,M,M))

MeshData of dimension 3 with 384 elements with periodicity = (true, true, true).

In [6]:
mesh = warp_mesh(uniform_mesh, reference_approximation, 0.1, L) 

MeshData of dimension 3 with 384 elements with periodicity = (true, true, true).

In [7]:
@time spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation);

  2.275349 seconds (4.66 M allocations: 309.740 MiB, 3.18% gc time, 85.91% compilation time)


In [8]:
@time results_path = save_project(conservation_law,
     spatial_discretization, exact_solution, form, (0.0, T),
     "results/advection_3d/", overwrite=true, clear=true);

  5.434095 seconds (14.76 M allocations: 928.601 MiB, 3.61% gc time, 99.20% compilation time: <1% of which was recompilation)


Using the `semidiscretize` function, we can create an `ODEProblem` object for use with OrdinaryDiffEq.jl's `solve` function. Here we've chosen the option for a weight-adjusted approximation of the mass matrix inverse from [Chan et al. (SISC 2017)](https://arxiv.org/abs/1608.03836), which allows for modal schemes to be applied efficiently at high order with explicit time-marching methods.

In [9]:
CFL = 0.1
h = L/(reference_approximation.N_p * spatial_discretization.N_e)^(1/3)
dt = CFL * h / sqrt(a[1]^2 + a[2]^2 + a[3]^2)

@time ode_problem = semidiscretize(conservation_law, spatial_discretization, exact_solution, form, (0.0, T))

 10.666794 seconds (5.35 M allocations: 490.579 MiB, 1.82% gc time, 98.06% compilation time: <1% of which was recompilation)


ODEProblem with uType Array{Float64, 3} and tType Float64. In-place: true
timespan: (0.0, 1.0)
u0: 35×1×384 Array{Float64, 3}:
[:, :, 1] =
  0.526841280496322
  0.23005944467759418
 -0.009290914470701711
 -0.012478658346043935
  0.0011386319281426142
  0.016956034917536326
  0.013327760319501633
 -0.006896153054356678
  0.0003458885535326595
 -0.03386825432222411
 -0.004124640875220375
 -0.0002157548364756823
 -0.00018607447383010376
  ⋮
  0.0008722770526677527
  0.001653483518393314
 -0.048480602708067935
 -0.02481140650378145
  0.0015031143912520525
 -0.0008232717452017709
 -0.0011703990564559778
  0.0004881093707532717
  0.032020378406276195
 -0.0022121807477622963
  0.0034048185582440708
  0.007831187284208134

[:, :, 2] =
  0.15469373738386977
  0.012677207600159956
 -0.02558718216154592
  0.0005493979794464938
  0.000771973567071769
 -0.0609469373541721
 -0.043071934900802814
  0.0012813045742525414
  0.0010511762058713436
 -0.0206785229748985
  0.002072454251621025
  0.000108278

In [ ]:
times = @snoopi_deep solve(ode_problem, CarpenterKennedy2N54(), adaptive=false, dt=dt,
    save_everystep=false)